# Optimization Of Algo Case 1

- Maximize Total Return from 2015-Now
- Minimize Sharpe Ratio: Avg Daily Return - Risk Free Rate(0.05) / STDEV(Daily Return)

- Paramters: 
    - Number of Stocks(NumStocks) int between (2,30)
    - How long to hold the stock(holdTime) int between (0,2)
    - Stop Loss float between (-0.01,-0.05)
    - How long to calculate the Percent Change from int between(2,7)

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pandas_market_calendars as mcal
import datetime
from Tools.Trade_Functions import *
import numpy as np
import mlflow
import optuna

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
experiment_counter=0

In [3]:
mlflow.set_experiment("Momentum Rider Tuning:V3")

<Experiment: artifact_location='file:///Users/atulkrishnan/Desktop/TradingAlgos/Stock_Picker_Algo/mlruns/486892596834591236', creation_time=1732514941966, experiment_id='486892596834591236', last_update_time=1732514941966, lifecycle_stage='active', name='Momentum Rider Tuning:V3', tags={}>

In [3]:
def calculate_mdd(return_vals):
    # Calculate the cumulative returns (wealth index)
    wealth_index = (1 + return_vals).cumprod()
    
    # Calculate the running maximum
    running_max = wealth_index.cummax()
    
    # Calculate drawdowns (difference between running max and current value)
    drawdowns = (wealth_index - running_max) / running_max
    
    # The maximum drawdown is the largest negative drawdown (i.e., most significant loss)
    mdd = drawdowns.min()  # This will give the maximum drawdown (most negative value)
    
    return mdd

In [4]:
def calculate_sortino_ratio(return_vals, risk_free_rate=0.05):
    # Calculate the excess returns over the risk-free rate
    excess_returns = return_vals - risk_free_rate / 252  # Convert annual risk-free rate to daily
    
    # Calculate downside deviation (negative returns only)
    downside_returns = excess_returns[excess_returns < 0]  # Only consider negative returns
    downside_deviation = np.std(downside_returns)  # Standard deviation of negative returns
    
    # Calculate the mean of excess returns
    mean_excess_return = np.mean(excess_returns)
    
    # Calculate the Sortino ratio (avoid division by zero if downside deviation is zero)
    if downside_deviation == 0:
        sortino_ratio = np.nan  # Return NaN if no downside deviation (e.g., no negative returns)
    else:
        sortino_ratio = mean_excess_return / downside_deviation
    
    return sortino_ratio

In [10]:


def yearly_plots(returnDf):
    """
    Saves a plot for each year in the range start_year to end_year.
    
    Args:
    - dataframe (pd.DataFrame): The DataFrame containing the data.
    - start_year (int): The starting year (inclusive).
    - end_year (int): The ending year (inclusive).
    - column_to_plot (str): The column to plot (e.g., 'ReturnVals').
    - date_column (str): The column containing dates (e.g., 'Date').
    
    Returns:
    - None
    """
    # Ensure the date column is in datetime format
    
    for year in range(2015, 2025 ):
        # Filter data for the specific year
        yearly_data = returnDf[returnDf["Date"].dt.year == year]
        
        if not yearly_data.empty:  # Only create plots for years with data
            plt.figure(figsize=(10, 5))
            plt.plot(
                yearly_data["Date"],
                yearly_data["ReturnVals"],
                label=f"{year}",
                color="blue"
            )
            plt.title(f"Retruns - {year}")
            plt.xlabel("Date")
            plt.ylabel("Returns")
            plt.legend()
            plt.grid(True, linestyle='--', alpha=0.6)
            
            # Save the plot
            plt.savefig(f"Visulizations/returns_{year}.png", dpi=300, bbox_inches='tight')
            plt.close()

# Example Usage:
# save_yearly_plots_simple(returnDf, 2015, 2024, 'ReturnVals', 'Date')


In [11]:
TimeInput = 2
NumStocks=15
holdTime=0
stopLoss=-0.01
cal=get_calendar()
#Date Range
DateRange=['2015-01-02','2024-11-07']
DateRange=[cal.index(DateRange[0]),cal.index(DateRange[1])]
TotalReturnList = list()



#Run Algorithim
for day in cal[DateRange[0]:DateRange[1]]:
    dailyReturn = pick_trade(RunDate=day,NumStocks=NumStocks,TimeInput=TimeInput,stopLoss=stopLoss,holdTime=holdTime,debug=False)
    TotalReturnList.append(dailyReturn)
returnDf = pd.DataFrame({'Date': pd.to_datetime(cal[DateRange[0]:DateRange[1]]),'Return':TotalReturnList})
returnDf['Date'] = pd.to_datetime(returnDf['Date'])

#Compound Retruns
M = 1
returnVal=[]
for i in returnDf["Return"]:
    M = M*(1+i)
    returnVal.append(M)
returnDf=pd.concat([returnDf,pd.DataFrame({"ReturnVals":returnVal})],axis=1)

#Calculate Metrics
SharpeRatio =(np.mean(returnDf["Return"]) * 252 - 0.05) / (np.std(returnDf["Return"]) * np.sqrt(252))
CumulativeReturn = returnDf.loc[len(returnDf)-1,"ReturnVals"]
max_drawdown = calculate_mdd(returnDf["Return"])
sortino_ratio = calculate_sortino_ratio(returnDf["Return"])


pLratio = len(returnDf["Return"][returnDf["Return"]>0])/ len(returnDf["Return"])

avg_loss= np.mean(returnDf["Return"][returnDf["Return"]<0])
avg_prof= np.mean(returnDf["Return"][returnDf["Return"]>0])



#Saving A plot
plt.figure(figsize=(15, 6))  # Set the figure size
plt.plot(returnDf['Date'], returnDf['ReturnVals'],  linestyle='-', color='blue', label="Return Vals")
plt.title("Return Values Over Time", fontsize=16)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Return Vals", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(fontsize=12)
plt.savefig("Visulizations/returns.png", dpi=300, bbox_inches='tight')
plt.close()
#LOG Paramters, Metrics and Artifacts





yearly_plots(returnDf)


for year in range(2015,2025):
    returnDf_year = returnDf[returnDf['Date'].dt.year == year]
    returnDf_year['Return'] = returnDf_year['Return'].astype(float)
   





UNABLE TO FIND:--- TFCF ----
UNABLE TO FIND:--- VEON ----
UNABLE TO FIND:--- BIDU ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- MDLZ ----
UNABLE TO FIND:--- SRCL ----
UNABLE TO FIND:--- TSCO ----
UNABLE TO FIND:--- DISH ----
UNABLE TO FIND:--- AMZN ----
UNABLE TO FIND:--- ALXN ----
UNABLE TO FIND:--- CHRW ----
UNABLE TO FIND:--- GOOG ----
UNABLE TO FIND:--- CHTR ----
UNABLE TO FIND:--- CSCO ----
UNABLE TO FIND:--- INTC ----
UNABLE TO FIND:--- BRCM ----
UNABLE TO FIND:--- MSFT ----
UNABLE TO FIND:--- NVDA ----
UNABLE TO FIND:--- CTSH ----
UNABLE TO FIND:--- ISRG ----
UNABLE TO FIND:--- STX ----
UNABLE TO FIND:--- EBAY ----
UNABLE TO FIND:--- BKNG ----
UNABLE TO FIND:--- ILMN ----
UNABLE TO FIND:--- AKAM ----
UNABLE TO FIND:--- TXN ----
UNABLE TO FIND:--- GOOGL ----
UNABLE TO FIND:--- WDC ----
UNABLE TO FIND:--- NFLX ----
UNABLE TO FIND:--- ADP ----
UNABLE TO FIND:--- GRMN ----
UNABLE TO FIND:--- EQIX ----
UNABLE TO FIND:--- ADBE ----
UNABLE TO FIND:--- AMGN ----
UNABLE TO FIN

UNABLE TO FIND:--- 1448062D ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- CTRX ----
UNABLE TO FIND:--- 1288453D ----
UNABLE TO FIND:--- KRFT ----
UNABLE TO FIND:--- 1448062D ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- CTRX ----
UNABLE TO FIND:--- 1288453D ----
UNABLE TO FIND:--- KRFT ----
UNABLE TO FIND:--- 1448062D ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- CTRX ----
UNABLE TO FIND:--- 1288453D ----
UNABLE TO FIND:--- KRFT ----
UNABLE TO FIND:--- 1448062D ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- CTRX ----
UNABLE TO FIND:--- 1288453D ----
UNABLE TO FIND:--- KRFT ----
UNABLE TO FIND:--- 1448062D ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- CTRX ----
UNABLE TO FIND:--- 1288453D ----
UNABLE TO FIND:--- KRFT ----
UNABLE TO FIND:--- 1448062D ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- CTRX ----
UNABLE TO FIND:--- 1288453D ----
UNABLE TO FIND:--- KRFT ----
UNABLE TO FIND:--- 1448062D ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIN

UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO 

UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- 1396924D ----
UNABLE TO FIND:--- 1396926D ----
UNABLE TO FIND:--- SNDK ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- 1396924D ----
UNABLE TO FIND:--- 1396926D ----
UNABLE TO FIND:--- SNDK ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- 1396924D ----
UNABLE TO FIND:--- 1396926D ----
UNABLE TO FIND:--- SNDK ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- 1396924D ----
UNABLE TO FIND:--- 1396926D ----
UNABLE TO FIND:--- SNDK ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- 1396924D ----
UNABLE TO FIND:--- 1396926D ----
UNABLE TO FIND:--- SNDK ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- 1396924D ----
UNABLE TO FIND:--- 1396926D ----
UNABLE TO FIND:--- SNDK ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- 1396924D ----
UNABLE TO FIND:--- 1396926D ----
UNABLE TO FIND:--- SNDK ----
UNABLE TO FIND:--- BBBYQ ----
UNABLE TO FIND:--- 1396924D ----
UNABLE TO FIND:--- 1396926D ----
UNABLE TO FIND:--- SNDK ----


UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO FIND:--- 1518855D ----
UNABLE TO 

...

UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- BKR ----
UNABLE TO FIND:--- CEG ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- GEHC ----
UNABLE TO FIND:--- GFS ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- PANW ----
UNABLE TO FIND:--- ROP ----
UNABLE TO FIND:--- WBD ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- BKR ----
UNABLE TO FIND:--- CEG ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- GEHC ----
UNABLE TO FIND:--- GFS ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- PANW ----
UNABLE TO FIND:--- ROP ----
UNABLE TO FIND:--- WBD ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- BKR ----
UNABLE TO FIND:--- CEG ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- GEHC ----
UNABLE TO FIND:--- GFS ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- PANW ----
UNABLE TO FIND:--- ROP ----
UNABLE TO FIND:--- WBD ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- BKR ----
UNABLE TO FIND:--- CEG ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- GEHC ----
UNABLE TO

UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- GEHC ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ROP ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- GEHC ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ROP ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- GEHC ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ROP ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- GEHC ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ROP ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- GEHC ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ROP ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- GEHC ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ROP ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- GEHC ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ROP ----
UNABLE

UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- GEHC ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ROP ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- GEHC ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ROP ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- GEHC ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ROP ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- GEHC ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ROP ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- GEHC ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ROP ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- GEHC ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ROP ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- GEHC ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ROP ----
UNABLE

UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE TO FIND:--- LIN ----
UNABLE TO FIND:--- ARM ----
UNABLE TO FIND:--- DASH ----
UNABLE T

/var/folders/5h/lv_gm99s337dww6tj437kv080000gn/T/ipykernel_86177/1014112083.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returnDf_year['Return'] = returnDf_year['Return'].astype(float)
/var/folders/5h/lv_gm99s337dww6tj437kv080000gn/T/ipykernel_86177/1014112083.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returnDf_year['Return'] = returnDf_year['Return'].astype(float)
/var/folders/5h/lv_gm99s337dww6tj437kv080000gn/T/ipykernel_86177/1014112083.py:63: SettingWithCopyWarning: 
A value is trying 

In [12]:
for year in range(2015,2025):
    returnDf_year = returnDf[returnDf['Date'].dt.year == year]
    returnDf_year['Return'] = returnDf_year['Return'].astype(float)
    print(str(year)+"Returns",np.prod(1 + returnDf_year['Return']))

2015Returns 1.0233360152202338
2016Returns 1.2074413703471767
2017Returns 1.222861998824381
2018Returns 0.9852072488929627
2019Returns 1.1616410420701708
2020Returns 0.9825527141552408
2021Returns 1.1613974927851
2022Returns 2.256404215332026
2023Returns 1.2527039733930698
2024Returns 1.196467692899393


/var/folders/5h/lv_gm99s337dww6tj437kv080000gn/T/ipykernel_86177/3710606710.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returnDf_year['Return'] = returnDf_year['Return'].astype(float)
/var/folders/5h/lv_gm99s337dww6tj437kv080000gn/T/ipykernel_86177/3710606710.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returnDf_year['Return'] = returnDf_year['Return'].astype(float)
/var/folders/5h/lv_gm99s337dww6tj437kv080000gn/T/ipykernel_86177/3710606710.py:3: SettingWithCopyWarning: 
A value is trying to 

In [13]:
np.mean(returnDf["Return"]) * 252

0.201418596573231

In [14]:
print("TimeInput", TimeInput,"\n")
print("NumStocks", NumStocks,"\n")
print("holdTime", holdTime,"\n")
print("stopLoss", stopLoss,"\n")

print("CumulativeReturn", CumulativeReturn,"\n")
print("SharpeRatio", SharpeRatio,"\n")
print("Profit_Loss_Ratio", pLratio,"\n")
print("avg_prof",avg_prof,"\n")
print("avg_loss", avg_loss,"\n")

print("Max_Draw_Down",max_drawdown,"\n")
print("sortino_ratio",sortino_ratio,"\n")

print("Anualized Std",np.std(returnDf["Return"])*np.sqrt(252))



TimeInput 2 

NumStocks 15 

holdTime 0 

stopLoss -0.01 

CumulativeReturn 6.673664713848743 

SharpeRatio 1.165480416774674 

Profit_Loss_Ratio 0.4864864864864865 

avg_prof 0.007094530106230697 

avg_loss -0.005168701122902766 

Max_Draw_Down -0.2760494840533315 

sortino_ratio 0.1574148841863746 

Anualized Std 0.12991946873913474


In [9]:
def objective(trial):
    global experiment_counter
    expirement_name = f"Expirement {experiment_counter}"
    
    with mlflow.start_run(run_name=expirement_name) as run:
        #Setup Paramter Variation
        TimeInput = trial.suggest_int('TimeInput', 2, 7)
        NumStocks=trial.suggest_int('NumStocks', 1, 5)
        holdTime=0
        stopLoss=trial.suggest_float('stopLoss',-0.05, -0.01)

        #Date Range
        DateRange=['2015-01-02','2024-11-07']
        DateRange=[cal.index(DateRange[0]),cal.index(DateRange[1])]
        TotalReturnList = list()

        

        #Run Algorithim
        for day in cal[DateRange[0]:DateRange[1]]:
            dailyReturn = pick_trade(RunDate=day,NumStocks=NumStocks,TimeInput=TimeInput,stopLoss=stopLoss,holdTime=holdTime,debug=False)
            TotalReturnList.append(dailyReturn)
        returnDf = pd.DataFrame({'Date': pd.to_datetime(cal[DateRange[0]:DateRange[1]]),'Return':TotalReturnList})
        returnDf['Date'] = pd.to_datetime(returnDf['Date'])

        #Compound Retruns
        M = 1
        returnVal=[]
        for i in returnDf["Return"]:
            M = M*(1+i)
            returnVal.append(M)
        returnDf=pd.concat([returnDf,pd.DataFrame({"ReturnVals":returnVal})],axis=1)

        #Calculate Metrics
        SharpeRatio =(np.mean(returnDf["Return"]) * 252 - 0.05) / (np.std(returnDf["Return"]) * np.sqrt(252))
        CumulativeReturn = returnDf.loc[len(returnDf)-1,"ReturnVals"]
        max_drawdown = calculate_mdd(returnDf["Return"])
        sortino_ratio = calculate_sortino_ratio(returnDf["Return"])

        
        pLratio = len(returnDf["Return"][returnDf["Return"]>0])/ len(returnDf["Return"])

        avg_loss= np.mean(returnDf["Return"][returnDf["Return"]<0])
        avg_prof= np.mean(returnDf["Return"][returnDf["Return"]>0])
        


        #Saving A plot
        plt.figure(figsize=(15, 6))  # Set the figure size
        plt.plot(returnDf['Date'], returnDf['ReturnVals'],  linestyle='-', color='blue', label="Return Vals")
        plt.title("Return Values Over Time", fontsize=16)
        plt.xlabel("Date", fontsize=12)
        plt.ylabel("Return Vals", fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.legend(fontsize=12)
        plt.savefig("returns.png", dpi=300, bbox_inches='tight')
        plt.close()
        #LOG Paramters, Metrics and Artifacts

        mlflow.log_param("TimeInput", trial.params['TimeInput'])
        mlflow.log_param("NumStocks", trial.params['NumStocks'])
        #mlflow.log_param("holdTime", trial.params['holdTime'])
        mlflow.log_param("stopLoss", trial.params['stopLoss'])

        mlflow.log_metric("CumulativeReturn", CumulativeReturn)
        mlflow.log_metric("SharpeRatio", SharpeRatio)
        mlflow.log_metric("Profit_Loss_Ratio", pLratio)
        mlflow.log_metric("avg_prof",avg_prof)
        mlflow.log_metric("avg_loss", avg_loss)
        
        mlflow.log_metric("Max_Draw_Down",max_drawdown)
        mlflow.log_metric("sortino_ratio",sortino_ratio)

        
        
        yearly_plots(returnDf)
        
        mlflow.log_artifact("returns.png", artifact_path="Visulizations")
        
        for year in range(2015,2025):
            returnDf_year = returnDf[returnDf['Date'].dt.year == year]
            returnDf_year['Return'] = returnDf_year['Return'].astype(float)
            mlflow.log_metric(str(year)+"Returns",np.prod(1 + returnDf_year['Return']))
            mlflow.log_artifact(f"returns_{year}.png", artifact_path="Visulizations")

            
        experiment_counter += 1
        
        return CumulativeReturn, SharpeRatio, sortino_ratio, max_drawdown